In [1]:
# if you want to use cuda, you can specify the ID of the device
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# and set the use_cuda parameter to True
use_cuda = True # otherwise, set it to False

In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

/ltstorage/home/nikishina/jupyterenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from tqdm import tqdm

In [4]:
from ru_detoxification_evaluation import load_model

## Reading input dataset and results of model inference

In [5]:
# We suppose that the input dataset is a dataframe with original toxic sentences 
# and/or with neutral refenrences
df = pd.read_csv('finalru_dataset_twocam_annotation_ratio.csv')
df = df.groupby(['vs', 'lang']).agg(list)

In [6]:
from collections import defaultdict
dic2sent = defaultdict(dict)

for (indices, values) in zip(df.index.values.tolist(), df.values.tolist()):
    dic2sent[indices[0]][indices[1]] = values[1]

In [7]:
vss = []
toxic_inputs = []
neutral_references = []

for k, i in dic2sent.items():
    if i.get("en", False) and i.get("ru", False):
        vss.append(k)
        toxic_inputs.append(i["en"])
        neutral_references.append(i["ru"])

toxic_inputs[15], neutral_references[15], vss[15]

(['Я заметил, что мой Firefox работает на Маке лучше и быстрее, чем Windows.',
  "Но есть компьютерные ноутбуки, которые легче, круче, спокойнее (беспилотный 1,1gz Pentium-M) и имеют более продолжительный срок службы батарей, чем любой Mac'Book.",
  'Мой дешевый ПК управляет WOW, честно говоря, одной из более простых игр, лучше, чем мой Мак.',
  'Я использовал MacBook Pros, потому что Windows работает быстрее и выглядит лучше на Mac, чем на Dells и HPs.',
  'Текущий ПК, на котором я создал себя, у него есть оборудование лучше и быстрее, чем у любого Мака для продажи, буквально каждый компонент быстрее и более высокого качества.',
  'Тот, который почти на 10x быстрее, чем мой шестилетний Леново, и тем не менее, окружающая среда, где я провел 95% своего времени (Outlook), исполнила BETTER и FASTER там, чем на моем Маке.',
  'Однако (и это часть, которая, возможно, устарела), каждый из них был в целом быстрее и лучше поддерживается на Windows, чем на Mac.',
  'Я никогда не сталкивался с п

## Meaning Preservation Score (SIM)

In [8]:
!pip install evaluate

In [9]:
from ru_detoxification_metrics import evaluate_cosine_similarity

In [10]:
import evaluate
rouge = evaluate.load('rouge')

In [11]:
meaning_model, meaning_tokenizer = load_model('ai-forever/sbert_large_nlu_ru', use_cuda=use_cuda, model_class=AutoModel)
#meaning_model, meaning_tokenizer = load_model('sentence-transformers/LaBSE', use_cuda=use_cuda, model_class=AutoModel)
#meaning_model, meaning_tokenizer = load_model('DeepPavlov/rubert-base-cased-sentence', use_cuda=use_cuda, model_class=AutoModel)

In [12]:
similarities = {}
for v, toxic_input, neutral_reference in zip(vss, toxic_inputs, neutral_references):
    similarity = evaluate_cosine_similarity(
        model = meaning_model,
        tokenizer = meaning_tokenizer,
        original_texts = toxic_input,
        rewritten_texts = neutral_reference,
        batch_size=32,
        verbose=True,
        )
    similarities[v] = np.mean(similarity.numpy())

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 48.03it/s]


In [13]:
similarities

{'ASP VS PHP_objA': 0.7100759,
 'ASP VS PHP_objB': 0.6438465,
 'Apple VS Google_objA': 0.7556896,
 'Apple VS Google_objB': 0.73192817,
 'Boeing VS Airbus_objA': 0.597751,
 'Boeing VS Airbus_objB': 0.54879576,
 'Canon VS Nikon_objA': 0.64517814,
 'Canon VS Nikon_objB': 0.65482277,
 'IELTS VS TOEFL_objA': 0.622118,
 'IELTS VS TOEFL_objB': 0.66442764,
 'Internet Explorer VS Firefox_objA': 0.4770833,
 'Internet Explorer VS Firefox_objB': 0.6885109,
 'Linux VS Windows_objA': 0.7757096,
 'Linux VS Windows_objB': 0.68880075,
 'MAC VS PC_objA': 0.5205584,
 'MAC VS PC_objB': 0.6935671,
 'NetBeans VS Eclipse_objA': 0.693168,
 'NetBeans VS Eclipse_objB': 0.65443814,
 'OpenGL VS Direct3D_objA': 0.6983212,
 'OpenGL VS Direct3D_objB': 0.6757315,
 'PHP VS Python_objA': 0.73066926,
 'PHP VS Python_objB': 0.7403809,
 'Python VS R_objA': 0.5057851,
 'Python VS R_objB': 0.6157971,
 'Xbox VS PlayStation_objA': 0.628526,
 'Xbox VS PlayStation_objB': 0.7090792,
 'БМВ VS Ауди_objA': 0.5982782,
 'БМВ VS Ауди_

In [14]:
print(f'Meaning preservation (SIM):  {np.mean(np.array(list(similarities.values())))}')
print(f'Meaning preservation (SIM):  {np.max(np.array(list(similarities.values())))}')
print(f'Meaning preservation (SIM):  {np.min(np.array(list(similarities.values())))}')

Meaning preservation (SIM):  0.6359953284263611
Meaning preservation (SIM):  0.834003746509552
Meaning preservation (SIM):  0.457640677690506


In [15]:
from pymystem3 import Mystem
m = Mystem()

def lemmatize_mystem(text):
    return [i for i in m.lemmatize(text) if not i.isspace()]

In [16]:
all_rouges1 = {}
all_rouges2 = {}
all_rougesl = {}
all_rougeslsum = {}

from nltk.stem.snowball import SnowballStemmer 

stemmer = SnowballStemmer("russian") 

for v, toxic_input, neutral_reference in tqdm(zip(vss, toxic_inputs, neutral_references), total=len(neutral_references)):
    rouge1 = []
    rouge2 = []
    rougel = []
    rouge_lsum = []
    for a in toxic_input:
        for b in neutral_reference:
            one = rouge.compute(predictions=[a], references=[b], tokenizer=lambda x: lemmatize_mystem(x))
            rouge1.append(one['rouge1'])
            rouge2.append(one['rouge2'])
            rougel.append(one['rougeL'])
            rouge_lsum.append(one['rougeLsum'])
    all_rouges1[v] = np.mean(np.array(rouge1))
    all_rouges2[v] = np.mean(np.array(rouge2)) 
    all_rougesl[v] = np.mean(np.array(rougel))
    all_rougeslsum[v] = np.mean(np.array(rouge_lsum))

100%|███████████████████████████████████████████| 76/76 [07:12<00:00,  5.69s/it]


In [17]:
print(f'Meaning preservation (SIM):  {np.mean(np.array(list(all_rouges1.values())))}')
print(f'Meaning preservation (SIM):  {np.mean(np.array(list(all_rouges2.values())))}')
print(f'Meaning preservation (SIM):  {np.mean(np.array(list(all_rougesl.values())))}')
print(f'Meaning preservation (SIM):  {np.mean(np.array(list(all_rougeslsum.values())))}')
all_rouges1, all_rouges2, all_rougesl, all_rougeslsum

Meaning preservation (SIM):  0.25681736464068694
Meaning preservation (SIM):  0.04620687829216794
Meaning preservation (SIM):  0.21178428064555757
Meaning preservation (SIM):  0.21178428064555757


({'ASP VS PHP_objA': 0.18866598072832866,
  'ASP VS PHP_objB': 0.280879156072215,
  'Apple VS Google_objA': 0.3045665601338828,
  'Apple VS Google_objB': 0.2635797224036012,
  'Boeing VS Airbus_objA': 0.2105177368839954,
  'Boeing VS Airbus_objB': 0.21070355237128863,
  'Canon VS Nikon_objA': 0.1823317013460677,
  'Canon VS Nikon_objB': 0.16778317776217286,
  'IELTS VS TOEFL_objA': 0.24814168818616109,
  'IELTS VS TOEFL_objB': 0.32082396164006527,
  'Internet Explorer VS Firefox_objA': 0.26312769360321386,
  'Internet Explorer VS Firefox_objB': 0.320808288835513,
  'Linux VS Windows_objA': 0.2737978641924108,
  'Linux VS Windows_objB': 0.21908189817579046,
  'MAC VS PC_objA': 0.2310459069817689,
  'MAC VS PC_objB': 0.18540057607649887,
  'NetBeans VS Eclipse_objA': 0.2952995465627825,
  'NetBeans VS Eclipse_objB': 0.12757307557349395,
  'OpenGL VS Direct3D_objA': 0.23428913831553305,
  'OpenGL VS Direct3D_objB': 0.2678159706042056,
  'PHP VS Python_objA': 0.202179871053949,
  'PHP VS P